In [1]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

import pickle
import activation_function as af

In [2]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [3]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
img = x_train[0]
label = t_train[0]

In [4]:
print(label)
print(img.shape)
img = img.reshape(28, 28)
print(img.shape)
img_show(img)

5
(784,)
(28, 28)


In [5]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(
        normalize=True,
        flatten=True,
        one_hot_label=False
    )
    return x_test, t_test

In [6]:
def init_network():
    with open("../models/sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
        
    return network

In [7]:
def predict(network, x):
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]
    
    a1 = np.dot(x, W1) + b1
    z1 = af.sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = af.sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = af.softmax(a3)
    return y

In [8]:
# サンプルモデルを利用したテストデータの判定
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1
        
print("Accuracy:" + str(float(accuracy_cnt / len(x))))

Accuracy:0.9352


In [10]:
# 重みの形状出力
x, _ = get_data()
network = init_network()
W1, W2, W3 = network["W1"], network["W2"], network["W3"]
print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


In [13]:
# バッチ処理を利用したケース
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print("Accuracy:" + str(float(accuracy_cnt / len(x))))

Accuracy:0.9352
